# RNN을 이용한 텍스트 생성
    '경마장에 있는 말이 뛰고 있다‘
    '그의 말이 법이다‘
    '가는 말이 고와야 오는 말이 곱다'

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [6]:
text = '''경마장에 있는 말이 뛰고 있다.
그의 말이 법이다.
가는 말이 고와야 오는 말이 곱다.'''


In [21]:
text.split('\n')

['경마장에 있는 말이 뛰고 있다.', '그의 말이 법이다.', '가는 말이 고와야 오는 말이 곱다.']

텍스트 전처리

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [8]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [9]:
# t.word_index에는 0 index가 없다

t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [10]:
# 단어 집합 크기 - Keras Tokenizer의 시작 인덱스가 1이기 때문
vocab_size = len(t.word_index) + 1
vocab_size

12

In [11]:
t.texts_to_sequences(['그의 말이 법이다.'])

[[6, 1, 7]]

In [13]:
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    print(encoded)

[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]


In [14]:
sequences = []
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequences.append(encoded[:i+1])

sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [16]:
max_len = max(len(seq) for seq in sequences)
max_len

6

- Padding
    - 전체 샘플 길이를 max_len에 맞추도록 0를 추가
    - pre 옵션을 주면 앞쪽을 채운다

In [17]:
# 가장 긴 timesteps(y 빼고 5개)에 shape을 맞춘다

sequences = pad_sequences(sequences, maxlen = max_len, padding = 'pre')
sequences[:5]

# 맨 마지막의 것이 y가 되고, 나머지는 x가 된다.

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]], dtype=int32)

In [18]:
X = sequences[:, :-1]
y = sequences[:, -1]

In [19]:
X.shape

(11, 5)

In [20]:
Y = to_categorical(y)       # 11까지 있으니 y의 갯수는 12개
Y[:3]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

- 모델 정의
    - Embedding
    - SimpleRNN
    - Dense

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [24]:
# Embedding = 4, SimpleRNN = 32
# Embedding은 11개의 단어(12개의 vocab size)를 4개로 줄이겠다는 의미

model = Sequential([
                    Embedding(vocab_size, 4, input_length=max_len-1),   # input_length는 timesteps
                    SimpleRNN(32),                                      # embedding에서 input_shape이 4차원이 된다
                    Dense(vocab_size, activation = 'softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 4)              48        
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                1184      
                                                                 
 dense_1 (Dense)             (None, 12)                396       
                                                                 
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


- 모델 설정/학습

In [25]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [26]:
hist = model.fit(X, Y, epochs = 200, verbose = 0)

In [27]:
hist.history['accuracy'][-1]

0.9090909361839294

- 모델 검증

In [28]:
from google.colab import files
up = files.upload()

Saving my_util.py to my_util.py


In [29]:
from my_util import sentence_generation

In [36]:
# 모델, 토크나이저, max_len, 현재 단어, 반복할 횟수

sentence_generation(model, t, max_len, '경마장에', 3)

'경마장에 말이 말이 뛰고'

In [37]:
sentence_generation(model, t, max_len, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'

### 모델 변화
- Embedding vector 갯수 : 2, 4, 6
- RNN 유닛 갯수 : 24, 32, 48

In [38]:
for n_embed in [2, 4, 6]:
    for n_unit in [24, 32, 48]:
        model = Sequential([
                            Embedding(vocab_size, n_embed, input_length=max_len -1),
                            SimpleRNN(n_unit),
                            Dense(vocab_size, activation = 'softmax')
        ])
        model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        hist = model.fit(X, Y, epochs = 200, verbose = 0)
        print('========================================')
        print(f'Embedding vector : {n_embed}, RNN_unit: {n_unit}')
        print(f"정확도: {hist.history['accuracy'][-1]: .4f}")
        print(sentence_generation(model, t, max_len, '경마장에', 4))
        print(sentence_generation(model, t, max_len, '그의', 2))
        print(sentence_generation(model, t, max_len, '가는', 5))
        print()

Embedding vector : 2, RNN_unit: 24
정확도:  0.9091
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector : 2, RNN_unit: 32
정확도:  0.8182
경마장에 말이 말이 뛰고 있다
그의 말이 말이
가는 말이 고와야 오는 말이 곱다

Embedding vector : 2, RNN_unit: 48
정확도:  0.9091
경마장에 말이 고와야 오는 말이
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector : 4, RNN_unit: 24
정확도:  0.9091
경마장에 있는 말이 뛰고 있다
그의 말이 고와야
가는 말이 고와야 오는 말이 곱다

Embedding vector : 4, RNN_unit: 32
정확도:  0.9091
경마장에 말이 고와야 오는 말이
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector : 4, RNN_unit: 48
정확도:  1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector : 6, RNN_unit: 24
정확도:  1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector : 6, RNN_unit: 32
정확도:  1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector : 6, RNN_unit: 48
정확도:  1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

